[View in Colaboratory](https://colab.research.google.com/github/mogamin/chainer-examples/blob/master/chainer_PotatoChips_Classification.ipynb)

In [2]:
# まずはデータのダウンロード、展開
#!curl -L "https://rebrand.ly/dllab2018-hackathon-cv" -o data.tar.gz
#!gzip -d -c data.tar.gz | tar xf -

total 3532048
drwxrwxr-x 5 1008 1008       4096 Jun 19 13:19 data
drwxr-xr-x 3 root root       4096 Aug  3 16:51 datalab
-rw-r--r-- 1 root root 3616797934 Aug  9 05:54 data.tar.gz
drwxr-xr-x 3 root root       4096 Aug  9 07:19 result


In [97]:
# デフォルトでcolabに入っていない使用するパッケージをインストール
#!pip install chainer
#!pip install chainercv

    100% |████████████████████████████████| 163kB 3.7MB/s 
  Running setup.py bdist_wheel for chainercv ... - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/d7/97/d3/1fbd4d4cd7aff64a3420daab1eaf32066e01baa6ec682f8481
Successfully built chainercv


In [0]:
import chainer
import chainer.links as L
import chainer.functions as F
from chainer.datasets import LabeledImageDataset
from chainer.datasets import TransformDataset

In [0]:
from chainercv.transforms import resize

# 画像データの前処理変換
class ImageTransform(object):
    def __call__(self, in_data):
        img, label = in_data
        #_, H, W = img.shape
        #img = random_sized_crop(img)
        img = resize(img, (224, 224))
        #img = random_flip(img, x_random=True)
        return img, label

In [0]:
# データのロード
train = LabeledImageDataset('data/train/train_labels.txt', root='data/train/images')
train = TransformDataset(train, ImageTransform())
valid = LabeledImageDataset('data/valid/valid_labels.txt', root='data/valid/images')
valid = TransformDataset(valid, ImageTransform())

In [0]:
class MLP(chainer.Chain):
  def __init__(self, n_mid_units=5, n_class=9):
    super().__init__()
    with self.init_scope():
        self.fc1 = L.Linear(None, n_mid_units)
        self.fc2 = L.Linear(None, n_mid_units)
        self.fc3 = L.Linear(None, n_class)
  
  def __call__(self, x):
    h = self.fc1(x)
    h = F.relu(h)
    h = self.fc2(h)
    h = F.relu(h)
    return h   
  

In [0]:
# イテレータの準備
BATCHSIZE = 100
train_iter = chainer.iterators.SerialIterator(train, BATCHSIZE)
valid_iter = chainer.iterators.SerialIterator(valid, BATCHSIZE, repeat=False, shuffle=False)

In [103]:
#
model = MLP(4096,9)
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

In [104]:
#
from chainer.training import extensions

EPOCH = 10
updater = chainer.training.StandardUpdater(train_iter, optimizer, device=-1)
trainer = chainer.training.Trainer(updater, (EPOCH, 'epoch'), out='result/potato')
trainer.extend(extensions.Evaluator(valid_iter, model, device=-1))
trainer.extend(extensions.LogReport(trigger=(1,'epoch')))
trainer.extend(extensions.PrintReport(['epoch','main/accuracy','validation/main/accuracy','main/loss','validation/main/loss','elapsed_time']), trigger=(1,'epoch'))
trainer.extend(extensions.ProgressBar(update_interval=10))
trainer.run()

Exception in main training loop: __call__() takes 2 positional arguments but 3 were given
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/chainer/training/trainer.py", line 306, in run
    update()
  File "/usr/local/lib/python3.6/dist-packages/chainer/training/updaters/standard_updater.py", line 149, in update
    self.update_core()
  File "/usr/local/lib/python3.6/dist-packages/chainer/training/updaters/standard_updater.py", line 160, in update_core
    optimizer.update(loss_func, *in_arrays)
  File "/usr/local/lib/python3.6/dist-packages/chainer/optimizer.py", line 650, in update
    loss = lossfun(*args, **kwds)
Will finalize trainer extensions and updater before reraising the exception.


TypeError: ignored